Data source

1. linkedin
2. Jobsdb(done)
3. Jobtopgun(ongoing)
4. Jobbkk
5. Jobthai

Methodology

1. Scrap Data from various website
2. collect data into database(spreadsheet) or clouddrive using Git version Control
3. write scipt to collect Jobs in every day and store in GeneratorExit
4. visualize the data using Power Bi or looker

In [5]:
# Import packages
import bs4
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException

## Selenium Test

In [6]:
driver_path = r'C:\Program Files (x86)\chromedriver-win64\chromedriver.exe'

def create_driver():
    options = Options()
    options.add_argument("--start-maximized")  # Start the browser maximized
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
    return driver


# Scraping Data from Jobsdb Data

In [7]:
# Define the base URL and the page URL
base_url = 'https://th.jobsdb.com'
search_data = '/data-jobs'
sec_page_url = '?page=2'

# Fetch the webpage content
page1 = requests.get(urljoin(base_url, search_data))

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(page1.content, 'html.parser')

In [8]:
box = soup.find('div',{'class':'_1decxdv0 _110qf3s4y _110qf3s4w'})

In [9]:
company_name = soup.find('a', {'data-automation': 'jobCompany'})
position = soup.find('a', {'data-automation':'jobTitle'})
location = soup.find('a', {'data-automation': 'jobLocation'})
industry = soup.find('a',{'data-automation':'jobClassification'})
date = soup.find('span',{'data-automation':'jobListingDate'})

In [10]:
company_name.text

'Universal Food Public Company Limited'

In [11]:
position.text

'Data Analyst'

In [12]:
location.text

'Bangkok'

In [13]:
industry.text

'(Accounting)'

In [14]:
date.text

'4h ago'

In [15]:
print(company_name.text)
print(position.text)
print(location.text)
print(industry.text)
print(date.text)

Universal Food Public Company Limited
Data Analyst
Bangkok
(Accounting)
4h ago


In [16]:
company_name_list = []
position_list = []
location_list = []

## Scrap one page to test

In [17]:
# Extract job titles
job_title_elements = soup.find_all('a', {'data-automation':'jobTitle'})
job_titles = [div.get_text(strip=True) for div in job_title_elements]

# Extract company names
company_name_elements = soup.find_all('a', {'data-automation': 'jobCompany'})
company_names = [a.get_text(strip=True) for a in company_name_elements]

# Extract Locations
location_elements = soup.find_all('a', {'data-automation': 'jobLocation'})
location = [a.get_text(strip=True) for a in location_elements]

# Extract industry
industry_elements = soup.find_all('a',{'data-automation':'jobClassification'})
industry = [a.get_text(strip=True) for a in industry_elements]

# Extract time
time_elements = soup.find_all('span',{'data-automation':'jobListingDate'})
time = [a.get_text(strip=True) for a in time_elements]

# Combine job titles and company names into a list of tuples
data = list(zip(job_titles, company_names, location, industry,time))

# Create DataFrame
data_jobs = pd.DataFrame(data, columns=['job_title', 'company_name','location','industry','time'])

In [18]:
data_jobs.head(3)

,job_title,company_name,location,industry,time
0,Data Analyst,Universal Food Public Company Limited,Bangkok,(Accounting),4h ago
1,Data Analyst,THAI UNION GROUP PCL.,Phaya Thai,(Information & Communication Technology),22h ago
2,Data Analytics,PTG Energy Public Company Limited,Bangkok,(Information & Communication Technology),1d ago


In [19]:
all_jobs = []

page_number = 1
previous_content = ""

while True:
    url = f"https://th.jobsdb.com/data-jobs?page={page_number}"
    response = requests.get(url)

    if response.status_code != 200:
        print("Failed to retrieve page")
        break

    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract job titles
    job_title_elements = soup.find_all('a', {'data-automation':'jobTitle'})
    job_titles = [div.get_text(strip=True) for div in job_title_elements]

    # Extract company names
    company_name_elements = soup.find_all('a', {'data-automation': 'jobCompany'})
    company_names = [a.get_text(strip=True) for a in company_name_elements]

    # Extract locations
    location_elements = soup.find_all('a', {'data-automation': 'jobLocation'})
    locations = [a.get_text(strip=True) for a in location_elements]

    # Extract industry
    industry_elements = soup.find_all('a',{'data-automation':'jobClassification'})
    industries = [a.get_text(strip=True) for a in industry_elements]

    # Extract time (job posting date)
    time_elements = soup.find_all('span',{'data-automation':'jobListingDate'})
    times = [a.get_text(strip=True) for a in time_elements]

    # Check if extraction was successful
    if len(job_titles) == 0 or len(company_names) == 0 or len(locations) == 0 or len(industries) == 0 or len(times) == 0:
        print(f"No new jobs found on page {page_number}, stopping.")
        break

    # Combine data into tuples and append to the all_jobs list
    page_data = list(zip(job_titles, company_names, locations, industries, times))
    all_jobs.extend(page_data)

    # Compare content to determine if the page is the same
    current_content = "".join(job_titles)
    if current_content == previous_content:
        print(f"No new content found on page {page_number}, stopping the loop.")
        break

    # Print success message
    print(f"Successfully scraped page {page_number}")

    previous_content = current_content
    page_number += 1

# Create DataFrame from all collected data
jobsdb_data = pd.DataFrame(all_jobs, columns=['job_title', 'company_name', 'location', 'industry', 'time'])

Successfully scraped page 1
Successfully scraped page 2
Successfully scraped page 3
Successfully scraped page 4
Successfully scraped page 5
Successfully scraped page 6
Successfully scraped page 7
Successfully scraped page 8
Successfully scraped page 9
Successfully scraped page 10


KeyboardInterrupt: 

In [ ]:
jobsdb_data.shape

NameError: name 'jobsdb_data' is not defined

In [ ]:
jobsdb_data.isnull().sum()

NameError: name 'jobsdb_data' is not defined

In [ ]:
jobsdb_data['company_name'].value_counts()

company_name
Agoda Services Co., Ltd.                                    326
PERSOLKELLY HR Services Recruitment (Thailand) Co., Ltd.    158
Company Confidential                                        117
Western Digital (Thailand) Co., Ltd.                         89
TikTok Pte. Ltd.                                             80
                                                           ... 
NISSAN MOTOR ASIA PACIFIC (THAILAND) CO., LTD.                1
Syaqua Siam Co., Ltd.                                         1
SPIE Global Services Energy (Thailand) Co., Ltd.              1
Herba Bangkok Sociedad Limitada                               1
Thai Christian School (TCS)                                   1
Name: count, Length: 1893, dtype: int64

In [ ]:
jobsdb_data['location'].value_counts()

location
Bangkok                       3327
Chon Buri                      201
Pathum Wan                     170
Samut Prakan                   168
Sathon                         144
                              ... 
Mueang Nakhon Si Thammarat       1
Chai Prakan                      1
Mueang Nakhon Ratchasima         1
Nakhon Luang                     1
Phetchabun                       1
Name: count, Length: 158, dtype: int64

In [ ]:
jobsdb_data['location'].nunique()

158

In [ ]:
jobsdb_data['location'].unique().tolist()

['Bang Rak',
 'Bangkok',
 'Samut Prakan',
 'Chatuchak',
 'Phra Khanong',
 'Phaya Thai',
 'Bang Kho Laem',
 'Ratchathewi',
 'Pathum Wan',
 'Lak Si',
 'Yan Nawa',
 'Bang Na',
 'Rayong',
 'Mueang Samut Prakan',
 'Khlong Toei',
 'Mueang Nonthaburi',
 'Nonthaburi',
 'Bang Kapi',
 'Lat Phrao',
 'Sathon',
 'Huai Khwang',
 'Bangkok Metropolitan Region',
 'Chon Buri',
 'Din Daeng',
 'Vadhana',
 'Thung Khru',
 'Bang Yai',
 'Bang Sao Thong',
 'Don Mueang',
 'Pathum Thani',
 'Saphan Sung',
 'Ayutthaya',
 'Dusit',
 'Wang Noi',
 'Chachoengsao',
 'Pak Kret',
 'Bangkok Noi',
 'Bang Phlat',
 'Suan Luang',
 'Sung Noen',
 'Nakhon Ratchasima',
 'Khon Kaen',
 'Bueng Kum',
 'Prawet',
 'Sattahip',
 'Phasi Charoen',
 'Thanyaburi',
 'Khlong San',
 'Others',
 'Surin',
 'Si Maha Phot',
 'Prachin Buri',
 'Min Buri',
 'Khlong Luang',
 'Mueang Pathum Thani',
 'Sam Phran',
 'Nakhon Pathom',
 'Phuket',
 'Mueang Lamphun',
 'Lamphun',
 'Lat Krabang',
 'Bang Phli',
 'Ban Chang',
 'Si Racha',
 'Pluak Daeng',
 'Samut Song

## Jobtopgun

In [ ]:
# Define the base URL and the page URL
base_url = 'https://www.jobtopgun.com/en'
search_data = '/jobs?keywords=data/data-jobs'
sec_page_url = '&page=2'

# Fetch the webpage content
jobtopgun = requests.get(urljoin(base_url, search_data))

# Parse the HTML content using BeautifulSoup
soup_topgun = BeautifulSoup(jobtopgun.content, 'html.parser')

In [ ]:
# Data information to scrap
jobtopgun_company_name = soup_topgun.find('span',{'class':'text-foreground'})

jobtopgun_jobtitle = soup_topgun.find('span',{'class':'text-lg font-medium text-sub-primary'})

jobtopgun_detailed_box = soup_topgun.find('div', {'class': 'flex flex-row gap-2'})

In [ ]:
jobtopgun_detailed_box = soup_topgun.find_all('div', {'class': 'flex flex-row gap-2'})

In [ ]:
detail_box_texts = [i.get_text() for i in jobtopgun_detailed_box]

In [ ]:
detail_box_texts

['สามารถเจรจาต่อรองได้',
 '0 - 5 ปี',
 'กทม. (ธนบุรี, บางแค)',
 'ปริญญาตรี',
 '25,000 - 30,000 บาท/เดือน',
 '2 - 5 ปี',
 'กทม. (พญาไท)',
 'ปริญญาตรีหรือสูงกว่า',
 'สามารถเจรจาต่อรองได้',
 '0 - 2 ปี',
 'สมุทรปราการ',
 'ปริญญาตรี',
 'สามารถเจรจาต่อรองได้',
 '4 - 8 ปี',
 'กรุงเทพและปริมณฑล',
 'ปริญญาตรี',
 'สามารถเจรจาต่อรองได้',
 '0 - 5 ปี',
 'นนทบุรี',
 'ปริญญาตรีหรือสูงกว่า',
 'สามารถเจรจาต่อรองได้',
 '1 - 7 ปี',
 'ชลบุรี',
 'ปริญญาตรีหรือสูงกว่า',
 'สามารถเจรจาต่อรองได้',
 '8 - 9 ปี',
 'สมุทรปราการ',
 'ปริญญาตรีหรือสูงกว่า',
 'สามารถเจรจาต่อรองได้',
 '8 - 10 ปี',
 'สมุทรปราการ',
 'ปริญญาตรีหรือสูงกว่า',
 'สามารถเจรจาต่อรองได้',
 '5 - 7 ปี',
 'สมุทรปราการ',
 'ปริญญาตรีหรือสูงกว่า',
 'ขึ้นอยู่กับคุณสมบัติและประสบการณ์',
 '1 ปี',
 'นนทบุรี',
 'ประกาศนียบัตรวิชาชีพ (ปวช.) หรือ ประกาศนียบัตรวิชาชีพชั้นสูง (ปวส.)',
 'สามารถเจรจาต่อรองได้',
 '5 - 10 ปี',
 'กทม. (คลองเตย)',
 'ปริญญาโท',
 'สามารถเจรจาต่อรองได้',
 '4 - 12 ปี',
 'กรุงเทพและปริมณฑล',
 'ปริญญาตรีหรือสูงกว่า',
 'สามารถเจรจาต่อรองได

In [ ]:
salary = []
for i in range(0, len(detail_box_texts), 4):
    salary.append(detail_box_texts[i])

experience = []
for i in range(1, len(detail_box_texts), 4):
    experience.append(detail_box_texts[i])

location = []
for i in range(2, len(detail_box_texts), 4):
    location.append(detail_box_texts[i])

education = []
for i in range(3, len(detail_box_texts), 4):
    education.append(detail_box_texts[i])

In [ ]:
box_list = [salary, experience, location, education]

# Print the length of each list
for lst in box_list:
    print(len(lst))

30
30
30
30


In [ ]:
jobtopgun_data = pd.DataFrame({'salary': salary,
                               'experience':experience,
                               'location':location,
                               'education':education})

In [ ]:
jobtopgun_data.head(3)

,salary,experience,location,education
0,สามารถเจรจาต่อรองได้,0 - 5 ปี,"กทม. (ธนบุรี, บางแค)",ปริญญาตรี
1,"25,000 - 30,000 บาท/เดือน",2 - 5 ปี,กทม. (พญาไท),ปริญญาตรีหรือสูงกว่า
2,สามารถเจรจาต่อรองได้,0 - 2 ปี,สมุทรปราการ,ปริญญาตรี


## Scrape Jobtopgun

In [ ]:
driver_jobtopgun = create_driver()

driver_jobtopgun.get("https://www.jobtopgun.com/en/jobs?keywords=data%2Fdata-jobs&view=252263-16")
print("Page title",driver_jobtopgun.title)
element = driver_jobtopgun.find_elements(By.XPATH, '//*[@id="scrollable-job-cards-container"]/a[1]/div[1]')
element.click()
time.sleep(5)


driver_jobtopgun.close()


Page title Find Jobs in Thailand, Online Job Search Site | JOBTOPGUN.COM


AttributeError: 'list' object has no attribute 'click'

In [ ]:
# Create the WebDriver instance
driver_jobtopgun = create_driver()

# Navigate to the webpage
driver_jobtopgun.get("https://www.jobtopgun.com/en/jobs?keywords=data%2Fdata-jobs&view=252263-16")

# Print the page title
print("Page title:", driver_jobtopgun.title)

# Initialize WebDriverWait
wait = WebDriverWait(driver_jobtopgun, 10)

try:
    # Loop through the elements from a[1] to a[30]
    for i in range(1, 31):
        # Construct the XPath for the current element
        xpath = f'//*[@id="scrollable-job-cards-container"]/a[{i}]/div[1]'
        try:
            # Wait for the element to be clickable
            element = wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
            # Click the element
            element.click()
            
            # Optionally, wait for some condition or scrape data after clicking
            print(f"Clicked element a[{i}]")
            
            # Wait for 5 seconds
            time.sleep(3)
            
            # Optional: If clicking the element loads new content and you want to go back
            # driver_jobtopgun.back()
            # Wait for the page to reload
            # wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="scrollable-job-cards-container"]/a/div[1]')))
            
        except Exception as e:
            print(f"Failed to click element a[{i}]: {e}")

finally:
    # Close the browser
    driver_jobtopgun.quit()


Page title: Find Jobs in Thailand, Online Job Search Site | JOBTOPGUN.COM
Clicked element a[1]
Clicked element a[2]
Clicked element a[3]
Clicked element a[4]
Clicked element a[5]
Clicked element a[6]
Failed to click element a[7]: Message: element click intercepted: Element <div class="flex flex-1 flex-col gap-4 px-5 py-4">...</div> is not clickable at point (527, 582). Other element would receive the click: <div data-slot="content" data-open="true" data-arrow="true" data-placement="bottom-start" class="z-10 px-2.5 py-1 inline-flex flex-col items-center justify-center subpixel-antialiased outline-none box-border bg-content1 rounded-medium shadow-medium tt-content-66dad4c0-4f26-449f-8830-4e07df3463a8 w-[390px] !max-w-[calc(100dvw-40px)] translate-y-1 overflow-auto !border-1 !border-primary !px-[21px] !py-[16px] font-sarabun text-sm font-normal text-[#0C0C0C]">...</div>
  (Session info: chrome=128.0.6613.114)
Stacktrace:
	GetHandleVerifier [0x00BA8213+26163]
	(No symbol) [0x00B39CC4]
	(No

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time


# Create the WebDriver instance
driver_jobtopgun = create_driver()

# Navigate to the webpage
driver_jobtopgun.get("https://www.jobtopgun.com/en/jobs?keywords=data%2Fdata-jobs&view=252263-16")

# Print the page title
print("Page title:", driver_jobtopgun.title)

# Initialize WebDriverWait
wait = WebDriverWait(driver_jobtopgun, 10)

# Store the original window handle
original_window = driver_jobtopgun.current_window_handle

try:
    # Loop through the elements from a[1] to a[30]
    for i in range(1, 31):
        # Construct the XPath for the current element
        xpath = f'//*[@id="scrollable-job-cards-container"]/a[{i}]/div[1]'
        
        try:
            # Wait for the element to be clickable
            element = wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
            # Click the element
            element.click()
            
            # Switch to the new window/tab
            WebDriverWait(driver_jobtopgun.window_handles, 10)
            new_window = [window for window in driver_jobtopgun.window_handles if window != original_window][0]
            driver_jobtopgun.switch_to.window(new_window)
            
            # Optionally, wait for some condition or scrape data
            print(f"Clicked element job{i}] and switched to new tab")

            # Wait for 5 seconds
            time.sleep(5)

            
            driver_jobtopgun.close()
            
            # Switch back to the original window
            driver_jobtopgun.switch_to.window(original_window)

        except Exception as e:
            print(f"Failed to handle element a[{i}]: {e}")

finally:
    # Close the original window
    driver_jobtopgun.quit()


NameError: name 'create_driver' is not defined

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Create the WebDriver instance
driver_jobtopgun = webdriver.Chrome()  # Use the appropriate WebDriver (e.g., Chrome, Firefox)

# Navigate to the webpage
driver_jobtopgun.get("https://www.jobtopgun.com/en/jobs?keywords=data%2Fdata-jobs&view=252263-16")

# Print the page title
print("Page title:", driver_jobtopgun.title)

# Initialize WebDriverWait
wait = WebDriverWait(driver_jobtopgun, 10)

# Store the original window handle
original_window = driver_jobtopgun.current_window_handle

try:
    # Loop through the elements from a[1] to a[30]
    for i in range(1, 31):
        # Construct the XPath for the current element
        xpath = f'//*[@id="scrollable-job-cards-container"]/a[{i}]/div[1]'
        
        try:
            # Wait for the element to be clickable
            element = wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
            # Click the element
            element.click()
            
            # Wait for the new window/tab to open and switch to it
            wait.until(EC.number_of_windows_to_be(2))
            new_window = [window for window in driver_jobtopgun.window_handles if window != original_window][0]
            driver_jobtopgun.switch_to.window(new_window)
            
            # Print the current URL
            print(f"Current URL after clicking job [{i}]: {driver_jobtopgun.current_url}")
            
            # Wait for 5 seconds (or you can do some data scraping here)
            time.sleep(2)
            
            # Close the new window/tab
            driver_jobtopgun.close()
            
            # Switch back to the original window
            driver_jobtopgun.switch_to.window(original_window)

        except Exception as e:
            print(f"Failed to handle element a[{i}]: {e}")

finally:
    # Close the original window
    driver_jobtopgun.quit()


Page title: Find Jobs in Thailand, Online Job Search Site | JOBTOPGUN.COM
Current URL after clicking job [1]: https://www.jobtopgun.com/en/job/252263/16
Current URL after clicking job [2]: https://www.jobtopgun.com/en/job/1882/42
Current URL after clicking job [3]: https://www.jobtopgun.com/en/job/4718/331
Current URL after clicking job [4]: https://www.jobtopgun.com/en/job/83/1676
Current URL after clicking job [5]: https://www.jobtopgun.com/en/job/83/1650
Current URL after clicking job [6]: https://www.jobtopgun.com/en/job/24170/189
Current URL after clicking job [7]: https://www.jobtopgun.com/en/job/17668/925
Current URL after clicking job [8]: https://www.jobtopgun.com/en/job/17668/944
Current URL after clicking job [9]: https://www.jobtopgun.com/en/job/17668/929
Current URL after clicking job [10]: https://www.jobtopgun.com/en/job/20217/153
Current URL after clicking job [11]: https://www.jobtopgun.com/en/job/4400/719
Current URL after clicking job [12]: https://www.jobtopgun.com/

In [35]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Create the WebDriver instance
driver_jobtopgun = webdriver.Chrome()  # Use the appropriate WebDriver (e.g., Chrome, Firefox)

# Navigate to the webpage
driver_jobtopgun.get("https://www.jobtopgun.com/en/jobs?keywords=data%2Fdata-jobs&view=252263-16")

# Print the page title
print("Page title:", driver_jobtopgun.title)

# Initialize WebDriverWait
wait = WebDriverWait(driver_jobtopgun, 10)

# Store the original window handle
original_window = driver_jobtopgun.current_window_handle

# List to store job data
job_data = []

try:
    # Loop through the elements from a[1] to a[30]
    for i in range(1, 31):
        # Construct the XPath for the current element
        xpath = f'//*[@id="scrollable-job-cards-container"]/a[{i}]/div[1]'
        
        try:
            # Wait for the element to be clickable
            element = wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
            # Click the element
            element.click()
            
            # Wait for the new window/tab to open and switch to it
            wait.until(EC.number_of_windows_to_be(2))
            new_window = [window for window in driver_jobtopgun.window_handles if window != original_window][0]
            driver_jobtopgun.switch_to.window(new_window)
            
            # Scrape data from the job detail page
            try:
                job_title = driver_jobtopgun.find_element(By.XPATH, '//h1[@class="font-medium text-sub-primary text-lg"]').text
                company_name = driver_jobtopgun.find_element(By.XPATH, '//span[@class="flex-1 font-medium text-lg"]').text
                industry = driver_jobtopgun.find_element(By.XPATH,'//*[@id="job-details"]/div[1]/div/div[1]/div[1]/section/div[1]/span').text
                job_info = {
                    "job_title": job_title,
                    "company_name": company_name,
                    "industry" : industry,
                    "url": driver_jobtopgun.current_url
                }
                job_data.append(job_info)

                print(f"Scraped data for job [{i}]: {job_info}")
            except Exception as e:
                print(f"Failed to scrape data for job [{i}]: {e}")        
            
            driver_jobtopgun.close()
            
            driver_jobtopgun.switch_to.window(original_window)

        except Exception as e:
            print(f"Failed to handle element a[{i}]: {e}")

finally:
    # Close the original window
    driver_jobtopgun.quit()

# Output the scraped data
for job in job_data:
    print(job)


Page title: Find Jobs in Thailand, Online Job Search Site | JOBTOPGUN.COM
Scraped data for job [1]: {'job_title': 'Data Management', 'company_name': 'Thai Smile Bus Co.,Ltd.', 'industry': 'Transportation/Logistic', 'url': 'https://www.jobtopgun.com/en/job/252263/16'}
Scraped data for job [2]: {'job_title': 'Data Analysis', 'company_name': 'Casa Rocca Co.,Ltd', 'industry': 'Construction/Furnishings, Packaging, Trading/Import/Export, Graphic Design', 'url': 'https://www.jobtopgun.com/en/job/1882/42'}
Scraped data for job [3]: {'job_title': 'Data Analyst (Management trainee contract 1 year)', 'company_name': 'Mattel Bangkok Limited', 'industry': 'Toy', 'url': 'https://www.jobtopgun.com/en/job/4718/331'}
Scraped data for job [4]: {'job_title': 'Senior Data Analyst (สังกัดปัญญธารา)', 'company_name': 'CP ALL PUBLIC COMPANY LIMITED (General)', 'industry': 'Today', 'url': 'https://www.jobtopgun.com/en/job/83/1676'}
Scraped data for job [5]: {'job_title': 'Data Analyst', 'company_name': 'CP ALL

In [38]:
jobtopgun_data = pd.DataFrame(job_data)

In [39]:
jobtopgun_data

,job_title,company_name,industry,url
0,Data Management,"Thai Smile Bus Co.,Ltd.",Transportation/Logistic,https://www.jobtopgun.com/en/job/252263/16
1,Data Analysis,"Casa Rocca Co.,Ltd","Construction/Furnishings, Packaging, Trading/I...",https://www.jobtopgun.com/en/job/1882/42
2,Data Analyst (Management trainee contract 1 year),Mattel Bangkok Limited,Toy,https://www.jobtopgun.com/en/job/4718/331
3,Senior Data Analyst (สังกัดปัญญธารา),CP ALL PUBLIC COMPANY LIMITED (General),Today,https://www.jobtopgun.com/en/job/83/1676
4,Data Analyst,CP ALL PUBLIC COMPANY LIMITED (General),Today,https://www.jobtopgun.com/en/job/83/1650
5,Data Engineer,"Roojai Co.,Ltd.",Insurance,https://www.jobtopgun.com/en/job/24170/189
6,Security Manager (Data Center),Gulf Energy Development Public Company Limited,Yesterday,https://www.jobtopgun.com/en/job/17668/925
7,Manager - Data Center NOC (Samut Prakan),Gulf Energy Development Public Company Limited,Petroleum/Energy/Mining,https://www.jobtopgun.com/en/job/17668/944
8,Data Center Operations-Shift (Samut Prakan) (N...,Gulf Energy Development Public Company Limited,Yesterday,https://www.jobtopgun.com/en/job/17668/929
9,เจ้าหน้าที่คีย์ข้อมูล,"Sevenfive Distributor Co.,Ltd.",Retail/Wholesale,https://www.jobtopgun.com/en/job/20217/153
